In [1]:
import requests
import warnings, sys
import urllib
import itertools
from requests.exceptions import ConnectionError, HTTPError, Timeout, TooManyRedirects
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import traceback
import logging
import logging.handlers
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pyodbc
import sqlalchemy as sa
from sqlalchemy import create_engine, event
import json
import time as ti
import datetime
from datetime import datetime
import re
import os
import asyncio
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import ctypes
import threading

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
global ImportActive, StageActive, ProdActive
global stage, conn
conn = False
con = None
try:
    
    global ImportActive, StageActive, ProdActive
    logging.basicConfig(filename = 'CommonConfigManualLog.log',
                        filemode='a',
                        format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                        datefmt='%H:%M:%S',
                        level=logging.DEBUG)
    logging.info(f'\nLOG START: {datetime.now()}\n')
    
    def open_connection():
        global server, database, driver, dformat, connection, con
        with open(r"C:\Users\gmoye001\config\config.json", 'r') as fh:
            config = json.load(fh)
        server = config['server']
        database = config['database']
        driver = config['driver']
        dformat = config['dformat']
        connection = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes'
        con = pyodbc.connect(connection)
        return con
    
    def connection_test():
        cur = con.cursor()
        cur.execute("SELECT @@version")
        row = cur.fetchone()
        print("Connection established to: ",row[0])
        cur.close()
        con.commit()
        logging.info(f"\nConnection established to: {row[0]}\n")
        return 
        
    def close_connection():
        con.close()
        conn = False
        return
    
    def checktables(con, tbl):
        if stage == False:
            cur = con.cursor()
            query = "SELECT * FROM information_schema.Tables WHERE table_schema = 'GCV_PRD' and table_name = '%s'" % tbl
            cur.execute(query)
            output = cur.fetchall()
            if output == []:
                output = f'\n[GCV_PRD].{tbl} does not exist in the database and will need to be created\n'
                print(output, end = "\r")
                logging.info(output, end = "\r")
                cur.close()
                return False
            else:
                print(output, end = "\r")
                cur.close()
            return True
        else:
            cur = con.cursor()
            query = "SELECT * FROM information_schema.Tables WHERE table_schema = 'GCV_STG' and table_name = '%s'" % tbl
            cur.execute(query)
            output = cur.fetchall()
            if output == []:
                output = f'\n[GCV_STG].{tbl} does not exist in the database and will need to be created\n'
                print(output, end = "\r")
                logging.info(output, end = "\r")
                cur.close()
                return False
            else:
                print(output, end = "\r")
                cur.close()
            return True

    
    def checkbackups(con, tbl):
        if stage == False:
            cur = con.cursor()
            query = "SELECT * FROM information_schema.Tables WHERE table_schema = 'GCV_PRD' and table_name = '%s_backup'" % tbl
            cur.execute(query)
            output = cur.fetchall()
            if output == []:
                output = f'\n[GCV_PRD].{tbl}_backup does not exist in the database and will need to be created once parent table has data\n'
                print(output, end = "\r")
                logging.info(output)
                cur.close()
                return False
            else:
                print(f'\nCurrent backup for [GCV_PRD].{tbl} exists.', end = "\r")
                logging.info(f'\nCurrent backup for [GCV_PRD].{tbl} exists.\n')
                cur.close()
                return True
        else:
            cur = con.cursor()
            query = "SELECT * FROM information_schema.Tables WHERE table_schema = 'GCV_STG' and table_name = '%s_backup'" % tbl
            cur.execute(query)
            output = cur.fetchall()
            if output == []:
                output = f'\n[GCV_STG].{tbl}_backup does not exist in the database and will need to be created once parent table has data\n'
                print(output, end = "\r")
                logging.info(output)
                cur.close()
                return False
            else:
                print(f'\nCurrent backup for [GCV_STG].{tbl}', end = "\r")
                logging.info(f'\nCurrent backup for [GCV_STG].{tbl}\n')
                cur.close()
                return True
    
    def tablecontent(con, tbl):
        if stage == False:
            cur = con.cursor()
            query = "SELECT COUNT(*) FROM [GCV_PRD].[%s]" % tbl
            cur.execute(query)
            output = cur.fetchone()
            #print(output)
            if output[0] == 0:
                print(f'\n{output[0]} Existing records', end = "\r")
                logging.info(f'\n{output[0]} Existing records\n')
                cur.close()
                return False
            else:
                print(f'\n{output[0]} Existing records', end = "\r")
                logging.info(f'\n{output[0]} Existing records\n')
                cur.close()
                return True
        else:
            cur = con.cursor()
            query = "SELECT COUNT(*) FROM [GCV_STG].[%s]" % tbl
            cur.execute(query)
            output = cur.fetchone()
            #print(output)
            if output[0] == 0:
                print(f'\n{output[0]} Existing records', end = "\r")
                logging.info(f'\n{output[0]} Existing records\n')
                cur.close()
                return False
            else:
                print(f'\n{output[0]} Existing records', end = "\r")
                logging.info(f'\n{output[0]} Existing records\n')
                cur.close()
                return True
        
    def backup_data(con, tbl):
        if stage == False:
            if is_identity(con, tbl) == True:
                try:
                    bstart_time = datetime.now()
                    cur = con.cursor()
                    query = f"SET IDENTITY_INSERT [GCV_PRD].[%s_backup] ON; INSERT INTO [GCV_PRD].[%s_backup] ({fields}) SELECT {fields} FROM [GCV_PRD].[%s]; SET IDENTITY_INSERT [GCV_PRD].[%s_backup] OFF;" % (tbl, tbl, tbl, tbl)
                    print(f'\n{query}\n')
                    cur.execute(query)
                    con.commit()
                    cur.close()
                    bend_time = datetime.now()
                    print('\nBackup Duration: {}'.format(bend_time - bstart_time))
                    duration = bend_time - bstart_time
                    logging.info(f'\nBackup Duration: {duration}')
                    print("")
                    logging.info('\n')   
                except(Exception, pyodbc.DatabaseError) as e:
                    print("")
                    logging.info('\n')
                    print(e)
                    logging.exception("message")
                    cur.close()
                    con.rollback()
            else:
                try:
                    bstart_time = datetime.now()
                    cur = con.cursor()
                    query = "INSERT INTO [GCV_PRD].[%s_backup] SELECT * FROM [GCV_PRD].[%s]" % (tbl, tbl)
                    cur.execute(query)
                    con.commit()
                    cur.close()
                    bend_time = datetime.now()
                    print('\nBackup Duration: {}'.format(bend_time - bstart_time))
                    duration = bend_time - bstart_time
                    logging.info(f'\nBackup Duration: {duration}')
                    print("")
                    logging.info('\n')   
                except(Exception, pyodbc.DatabaseError) as e:
                    print("")
                    logging.info('\n')
                    print(e)
                    logging.exception("message")
                    cur.close()
                    con.rollback()
            return
        else:
            if is_identity(con, tbl) == True:
                try:
                    bstart_time = datetime.now()
                    cur = con.cursor()
                    query = f"SET IDENTITY_INSERT [GCV_STG].[%s_backup] ON; INSERT INTO [GCV_STG].[%s_backup] ({fields}) SELECT {fields} FROM [GCV_STG].[%s]; SET IDENTITY_INSERT [GCV_STG].[%s_backup] OFF;" % (tbl, tbl, tbl, tbl)
                    print(f'\n{query}\n')
                    cur.execute(query)
                    con.commit()
                    cur.close()
                    bend_time = datetime.now()
                    print('\nBackup Duration: {}'.format(bend_time - bstart_time))
                    duration = bend_time - bstart_time
                    logging.info(f'\nBackup Duration: {duration}')
                    print("")
                    logging.info('\n')   
                except(Exception, pyodbc.DatabaseError) as e:
                    print("")
                    logging.info('\n')
                    print(e)
                    logging.exception("message")
                    cur.close()
                    con.rollback()
            else:
                try:
                    bstart_time = datetime.now()
                    cur = con.cursor()
                    query = "INSERT INTO [GCV_STG].[%s_backup] SELECT * FROM [GCV_STG].[%s]" % (tbl, tbl)
                    cur.execute(query)
                    con.commit()
                    cur.close()
                    bend_time = datetime.now()
                    print('\nBackup Duration: {}'.format(bend_time - bstart_time))
                    duration = bend_time - bstart_time
                    logging.info(f'\nBackup Duration: {duration}')
                    print("")
                    logging.info('\n')   
                except(Exception, pyodbc.DatabaseError) as e:
                    print("")
                    logging.info('\n')
                    print(e)
                    logging.exception("message")
                    cur.close()
                    con.rollback()
            return
    
    def get_fields(con, tbl):
        global fields
        if stage == False:
            schema = 'GCV_PRD'
            try:
                cur = con.cursor()
                query = f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = '%s' AND TABLE_NAME = '%s'" % (schema, tbl)
                print(f'\n{query}\n')
                cur.execute(query)
                columns = cur.fetchall()
                columns = columns[0:]
                columns = list(zip(*columns))[0]
                fields = ", ".join(map(str, columns))
                print(f'\n{fields}\n')
                con.commit()
                cur.close()
                print("")
                logging.info('\n')   
            except(Exception, pyodbc.DatabaseError) as e:
                print("")
                logging.info('\n')
                print(e)
                logging.exception("message")
                cur.close()
                con.rollback()
            return fields
        else:
            schema = 'GCV_STG'
            try:
                cur = con.cursor()
                query = f"SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = '%s' AND TABLE_NAME = '%s'" % (schema, tbl)
                print(f'\n{query}\n')
                cur.execute(query)
                columns = cur.fetchall()
                columns = columns[0:]
                columns = list(zip(*columns))[0]
                fields = ", ".join(map(str, columns))
                print(f'\n{fields}\n')
                con.commit()
                cur.close()
                print("")
                logging.info('\n')   
            except(Exception, pyodbc.DatabaseError) as e:
                print("")
                logging.info('\n')
                print(e)
                logging.exception("message")
                cur.close()
                con.rollback()
            return fields
        
    def is_identity(con, tbl):
        if stage == False:
            try:
                cur = con.cursor()
                query = f"SELECT is_identity FROM sys.identity_columns WHERE object_id in ( SELECT id FROM sysobjects WHERE NAME = '%s')" % (tbl)
                print(f'\n{query}\n')
                cur.execute(query)
                output = cur.fetchone()
                if output is None:
                    print(f'\n{output}\n')
                    con.commit()
                    cur.close()
                    return False
                else:
                    output = output[0]
                    print(f'\n{output}\n')
                    con.commit()
                    cur.close()
                    return True
                print("")
                logging.info('\n')   
            except(Exception, pyodbc.DatabaseError) as e:
                print("")
                logging.info('\n')
                print(e)
                logging.exception("message")
                cur.close()
                con.rollback()
            return 
        else:
            try:
                cur = con.cursor()
                query = "SELECT is_identity FROM sys.identity_columns WHERE object_id in ( SELECT id FROM sysobjects WHERE NAME = '%s')" % (tbl)
                print(f'\n{query}\n')
                cur.execute(query)
                output = cur.fetchone()
                if output is None:
                    print(f'\n{output}\n')
                    con.commit()
                    cur.close()
                    return False
                else:
                    output = output[0]
                    print(f'\n{output}\n')
                    con.commit()
                    cur.close()
                    return True
                con.commit()
                cur.close()
                print("")
                logging.info('\n')   
            except(Exception, pyodbc.DatabaseError) as e:
                print("")
                logging.info('\n')
                print(e)
                logging.exception("message")
                cur.close()
                con.rollback()
            return
        
    def create_backuptable(con, tbl):
        if stage == False:
            try:
                bstart_time = datetime.now()
                cur = con.cursor()
                query = "SELECT * INTO [GCV_PRD].[%s_backup] FROM [GCV_PRD].[%s]" % (tbl, tbl)
                cur.execute(query)
                con.commit()
                cur.close()
                bend_time = datetime.now()
                logging.info(f'\n[GCV_PRD].{tbl} backup table has been created.\n')
                print("")
                logging.info('\n')   
            except(Exception, pyodbc.DatabaseError) as e:
                print("")
                logging.info('\n')
                print(e)
                logging.exception("message")
                cur.close()
                con.rollback()
                return False
            return True
        else:
            try:
                bstart_time = datetime.now()
                cur = con.cursor()
                query = "SELECT * INTO [GCV_STG].[%s_backup] FROM [GCV_STG].[%s]" % (tbl, tbl)
                cur.execute(query)
                con.commit()
                cur.close()
                bend_time = datetime.now()
                logging.info(f'\n[GCV_STG].{tbl} backup table has been created.\n')
                print("")
                logging.info('\n')   
            except(Exception, pyodbc.DatabaseError) as e:
                print("")
                logging.info('\n')
                print(e)
                logging.exception("message")
                cur.close()
                con.rollback()
                return False
            return True
        
    def backupcheck(con, tbl):
        global rowcount
        if stage == False:
            cur = con.cursor()
            query = "SELECT COUNT(*) FROM [GCV_PRD].[%s] UNION ALL SELECT COUNT(*) FROM [GCV_PRD].[%s_backup]" % (tbl, tbl)
            cur.execute(query)
            output = cur.fetchone()
            rowcount = []
            while output is not None:
                rowcount.append(output[0])
                output = cur.fetchone()
            print('\n# of records in each table: ', rowcount[0], recnum, end = "\r")
            logging.info(f'\n# of records in each table: {rowcount[0]}, {recnum}')
            con.commit()
            cur.close()
            if recnum == rowcount[0]:
                truncate_backup(con, tbl)
                get_fields(con, tbl)
                backup_data(con, tbl)
            elif recnum < rowcount[0]:
                truncate_backup(con, tbl)
                get_fields(con, tbl)
                backup_data(con, tbl)
            elif rowcount[0] == 0:
                pass
            return rowcount
        else:
            cur = con.cursor()
            query = "SELECT COUNT(*) FROM [GCV_STG].[%s] UNION ALL SELECT COUNT(*) FROM [GCV_STG].[%s_backup]" % (tbl, tbl)
            cur.execute(query)
            output = cur.fetchone()
            rowcount = []
            while output is not None:
                rowcount.append(output[0])
                output = cur.fetchone()
            print('\n# of records in each table: ', rowcount[0], recnum, end = "\r")
            logging.info(f'\n# of records in each table: {rowcount[0]}, {recnum}')
            con.commit()
            cur.close()
            if recnum == rowcount[0]:
                truncate_backup(con, tbl)
                get_fields(con, tbl)
                backup_data(con, tbl)
            elif recnum < rowcount[0]:
                truncate_backup(con, tbl)
                get_fields(con, tbl)
                backup_data(con, tbl)
            elif rowcount[0] == 0:
                pass
            return rowcount
    
    def truncate_table(con, tbl):
        if stage == False:
            try:
                cur = con.cursor()
                query = "TRUNCATE TABLE [GCV_PRD].[%s]" % tbl
                cur.execute(query)
                print(f'\n[GCV_PRD].{tbl} has been succesfully truncated to import new data.')
                logging.info(f'\n[GCV_PRD].{tbl} has been succesfully truncated to import new data.\n')
                con.commit()
                cur.close()
            except Exception as err:
                logging.exception("message")
                cur.close()
                con.rollback()
                #raise err
        else:
            try:
                cur = con.cursor()
                query = "TRUNCATE TABLE [GCV_STG].[%s]" % tbl
                cur.execute(query)
                print(f'\n[GCV_STG].{tbl} has been succesfully truncated to import new data.')
                logging.info(f'\n[GCV_STG].{tbl} has been succesfully truncated to import new data.\n')
                con.commit()
                cur.close()
            except Exception as err:
                logging.exception("message")
                cur.close()
                con.rollback()
                #raise err
            
    def truncate_backup(con, tbl):
        if stage == False:
            try:
                cur = con.cursor()
                query = "TRUNCATE TABLE [GCV_PRD].[%s_backup]" % tbl
                cur.execute(query)
                print(f'\n[GCV_PRD].{tbl}_backup has been succesfully truncated to create the next backup.')
                logging.info(f'\n[GCV_PRD].{tbl}_backup has been succesfully truncated to create the next backup.\n')
                con.commit()
                cur.close()
            except Exception as err:
                logging.exception("message")
                cur.close()
                con.rollback()
                #raise err
        else:
            try:
                cur = con.cursor()
                query = "TRUNCATE TABLE [GCV_STG].[%s_backup]" % tbl
                cur.execute(query)
                print(f'\n[GCV_STG].{tbl}_backup has been succesfully truncated to create the next backup.')
                logging.info(f'\n[GCV_STG].{tbl}_backup has been succesfully truncated to create the next backup.\n')
                con.commit()
                cur.close()
            except Exception as err:
                logging.exception("message")
                cur.close()
                con.rollback()
                #raise err

    def importdata(con, tbl):
        if stage == False:
            cur = con.cursor()
            engine = sa.create_engine(f'mssql+pyodbc://{server}/{database}?driver={dformat}', fast_executemany = True)
            #pd.io.sql._is_sqlalchemy_connectable(engine)
            df.to_sql(f'{tbl}', engine, index = False, if_exists = 'append', schema = 'GCV_PRD')
            cur.close()
            con.commit()
            return
        else:
            cur = con.cursor()
            engine = sa.create_engine(f'mssql+pyodbc://{server}/{database}?driver={dformat}', fast_executemany = True)
            #pd.io.sql._is_sqlalchemy_connectable(engine)
            df.to_sql(f'{tbl}', engine, index = False, if_exists = 'append', schema = 'GCV_STG')
            cur.close()
            con.commit()
            return
            
    def getData(CV):
        # convert to config file/table
        if stage == False:
            url = f'https://api.pwcinternal.com:7443/GlobalCVService/GlobalCVService.svc/cv/{CV}'
            with open(r'config\apiconnect.json') as f:
                headers = json.load(f)
        else:
            url = f'https://api-staging.pwcinternal.com:7443/GlobalCVService/GlobalCVService.svc/cv/{CV}'
            with open(r'config\apiconnectb.json') as f:
                headers = json.load(f)
                
        retry_strategy = Retry(total = 10, status_forcelist=[429, 413, 503], method_whitelist=["HEAD", "GET", "PUT", "DELETE", "OPTIONS", "TRACE"])
        adapter = HTTPAdapter(max_retries=retry_strategy)
        http = requests.Session()
        
        http.mount("https://", adapter)
        r = http.get(url, headers=headers, timeout = 100)
        rjson = r.json()
        keylist = ('URI','Categories','RelatedTerms')
        
        if 'ErrorMessage' in rjson:
            if rjson['ErrorMessage'] == 'CV does not exist in CVMaster List':
                print('CV Currently Missing from CVMaster List')
                return
        else:
            for key in keylist:
                rjson = [{k: v for k, v in d.items() if k != key} for d in rjson]
        
            global df, recnum
            df = pd.DataFrame(rjson)
            datelist = ('CreatedDate','ModifiedDate','EffectiveDate', 'HierarchyNodeEffectiveDate', 'HierarchyNodeExpirationDate','RelModifiedDate','ExpiryDate','Effective_Date','Expiration_Date','Created_Datetime','Last_Modified_Datetime')
            date_format = "%Y%m%d%H%M%S"
            
            with warnings.catch_warnings():
                warnings.simplefilter('ignore', FutureWarning)
                for date in enumerate(datelist): 
                    if date[1] in df:
                        
                        if df[date[1]].iteritems() != 'None':
                            if df[date[1]].str.contains('Z').items():
                                df[date[1]] = df[date[1]].str.replace("\.[0-9]*Z", "").str.replace("Z", "")
                        if df[date[1]].iteritems() != 'None':   
                            if df[date[1]].str.contains('-').items():
                                df[date[1]] = df[date[1]].str.replace("\W+", "")
                                
                        df[date[1]] = df[date[1]].mask(df[date[1]].str.len() > 14, df[date[1]].str[:-3])
                        df[date[1]] = pd.to_datetime(df[date[1]], format=date_format, errors = 'coerce')
                        
                    else:
                        continue
                    
            recnum = len(df.index)
            
            #display(df)
            #df.to_excel('output1.xlsx')
            
            if tablecontent(con, f'{CV}') == False:
                importdata(con, f'{CV}')
                print(f'\n{CV} has been updated with {recnum} records')
                logging.info(f'\n{CV} has been updated with {recnum} records\n')
            else:
                truncate_table(con, f'{CV}')
                importdata(con, f'{CV}')
                print(f'\n{CV} has been updated with {recnum} records')
                logging.info(f'\n{CV} has been updated with {recnum} records\n')
                
            #colnames = list(df)
            
            #display(colnames) 
             #[['Effective_Date','Expiration_Date','Created_Datetime','Last_Modified_Datetime']])
            
            #--fetch column names for table creation and datatypes
            #for colname, dt in itertools.product([df.columns],[df.dtypes]):
                #print(dt)
            #print('\n')
        
            return df
    
    ListExtS = ['All', 'Exit', 'Prod']
    ListExtP = ['All', 'Exit', 'Stage']
    
    def open_stage():
        with open(r'config\CVsStage.csv', 'r') as cv_config:
            optionsstage = cv_config.read().split(',')
        
        global StageSelect, ExecuteButton1
        
        optionsstage.extend(ListExtS)
        
        StageSelect = widgets.SelectMultiple(
        options = optionsstage,
        value = [optionsstage[0]],
        description='Select Stage CVs:',
        style = {'description_width': 'initial'},
        layout = Layout(width = 'auto', height = 'auto'),
        )
        
        ExecuteButton1 = widgets.Button(description='Import Selected CVs')
        output1 = widgets.Output()
        
        display(StageSelect, ExecuteButton1)
    
    def open_prod():
        with open(r'config\CVsProd.csv', 'r') as cv_config:
            optionsprod = cv_config.read().split(',')
            
        global ProdSelect, ExecuteButton2
        
        optionsprod.extend(ListExtP)
        
        ProdSelect = widgets.SelectMultiple(
        options = optionsprod,
        value = [optionsprod[0]],
        description='Select Prod CVs:',
        style = {'description_width': 'initial'},
        layout = Layout(width = 'auto', height = 'auto'),
        )
        
        ExecuteButton2 = widgets.Button(description='Import Selected CVs')
        output2 = widgets.Output()
        
        display(ProdSelect, ExecuteButton2)
        
        ProdSelect.layout.visibility = 'hidden'
        ExecuteButton2.layout.visibility = 'hidden'
    
    st = datetime.now()
    open_connection()
    connection_test()
    close_connection()
    open_stage()
    open_prod()
    
    def show_prod():
        ProdSelect.layout.visibility = 'visible'
        ExecuteButton2.layout.visibility = 'visible'
 
    def show_stage():
        StageSelect.layout.visibility = 'visible'
        ExecuteButton1.layout.visibility = 'visible'
    
    def hide_prod():
        ProdSelect.layout.visibility = 'hidden'
        ExecuteButton2.layout.visibility = 'hidden'
 
    def hide_stage():
        StageSelect.layout.visibility = 'hidden'
        ExecuteButton1.layout.visibility = 'hidden'
    
    def timedata():
        et = datetime.now()
        print('Total Execution Duration: {}'.format(et - st),'\n-Import Completed-')
        tt = et - st
        logging.info(f'\nTotal Execution Duration: {tt}\n')
        logging.info('\n-Import Completed-\n')
            
    def on_button1_clicked(b):
        global stage, conn
        hide_stage()
        
        CVsStage = list(StageSelect.value)
        if (('All' in CVsStage) or ('Prod' in CVsStage) or ('Exit' in CVsStage)) and (len(CVsStage) > 1):
            ctypes.windll.user32.MessageBoxW(0, "You cannot select multiple options for 'All', 'Exit', or 'Prod'", "Error:")
            show_stage()
        elif (CVsStage == ['All'] and len(CVsStage) == 1):
            CVsStage = optionsstage[:-3]
        elif (CVsStage == ['Exit']  and len(CVsStage) == 1):
            StageSelect.close()
            ExecuteButton1.close()
            timedata()
            os.system('cls')
        elif (CVsStage == ['Prod']  and len(CVsStage) == 1):
            show_prod()
        else:
            global stage, conn
            for urls in enumerate(CVsStage):
                stage = True
                conn = True
                open_connection()
                #getData(urls[1])
                #close_connection()
                if checktables(con, urls[1]) == False:
                    print("Moving to next table.\n", end = "\r")
                else:
                    if checkbackups(con, urls[1]) == False:
                        create_backuptable(con, urls[1])
                        createdbackup = True
                    else:
                        createdbackup = False
                    try:
                        gstart_time = datetime.now()
                        getData(urls[1])
                        if createdbackup == False:
                            backupcheck(con, urls[1])
                        else:
                            pass
                        close_connection()
                        stage = False
                        conn = False
                        gend_time = datetime.now()
                        print('\nDuration: {}'.format(gend_time - gstart_time))
                        logging.info('\nDuration: {}'.format(gend_time - gstart_time))
                        print("")
                    except (Exception, pyodbc.DatabaseError) as e:
                        print("")
                        print(e)
                        logging.exception('\n')
                        logging.exception("message")
                    for i in range(3, -1, -1):
                            print(f"{i} seconds until next table is imported ", end = "\r")
                            ti.sleep(1)
                        
                show_stage()
    
    ExecuteButton1.on_click(on_button1_clicked)
                
    def on_button2_clicked(b):
        hide_prod()
        global stage
        CVsProd = list(ProdSelect.value)
        if (('All' in CVsProd) or ('Prod' in CVsProd) or ('Exit' in CVsProd)) and (len(CVsProd) > 1):
            ctypes.windll.user32.MessageBoxW(0, "You cannot select multiple options for 'All', 'Exit', or 'Prod'", "Error:")
            show_prod()
        elif (CVsProd == ['All'] and len(CVsProd) == 1):
            CVsProd = optionsprod[:-3]
        elif (CVsProd == ['Exit'] and len(CVsProd) == 1):
            ProdSelect.close()
            ExecuteButton2.close()
            timedata()
            os.system('cls')
        elif (CVsProd == ['Stage'] and len(CVsProd) == 1):
            show_stage()
        else:
            for urls in enumerate(CVsProd):
                open_connection()
                stage = False
                conn = True
                #getData(urls[1])
                #close_connection()
                if checktables(con, urls[1]) == False:
                    print("Moving to next table.\n", end = "\r")
                else:
                    if checkbackups(con, urls[1]) == False:
                        create_backuptable(con, urls[1])
                        createdbackup = True
                    else:
                        createdbackup = False
                    try:
                        gstart_time = datetime.now()
                        getData(urls[1])
                        if createdbackup == False:
                            backupcheck(con, urls[1])
                        else:
                            pass
                        close_connection()
                        conn = False
                        gend_time = datetime.now()
                        print('\nDuration: {}'.format(gend_time - gstart_time))
                        logging.info('\nDuration: {}'.format(gend_time - gstart_time))
                        print("")
                    except (Exception, pyodbc.DatabaseError) as e:
                        print("")
                        print(e)
                        logging.exception('\n')
                        logging.exception("message")
                    for i in range(3, -1, -1):
                            print(f"{i} seconds until next table is imported ", end = "\r")
                            ti.sleep(1)
                        
                show_prod()

    ExecuteButton2.on_click(on_button2_clicked)
    
#exceptions 
except (Exception, pyodbc.DatabaseError) as error:
        print(error)
        logging.exception("message")
        pass
    
except requests.exceptions.HTTPError as errh:
    print("Http Error:",  errh)
    conn = False
    logging.exception("message")
    
except requests.exceptions.ConnectionError as errc:
    print("Error Connecting:", errc)
    conn = False
    logging.exception("message")
    
except requests.exceptions.Timeout as errt:
    print("Timeout Error:", errt)
    conn = False
    logging.exception("message")
    
except requests.exceptions.RequestException as erru:
    print("Unidentified Request Exception:", erru)
    conn = False
    logging.exception("message")
        
finally:
    logging.info(f'\nLOG END: {datetime.now()}\n')
    if conn == True:
        close_connection()
        cur.close()

Connection established to:  Microsoft SQL Server 2016 (SP2) (KB4052908) - 13.0.5026.0 (X64) 
	Mar 18 2018 09:11:49 
	Copyright (c) Microsoft Corporation
	Enterprise Edition (64-bit) on Windows Server 2016 Datacenter 10.0 <X64> (Build 14393: ) (Hypervisor)



SelectMultiple(description='Select Stage CVs:', index=(0,), layout=Layout(height='auto', width='auto'), option…

Button(description='Import Selected CVs', style=ButtonStyle())

SelectMultiple(description='Select Prod CVs:', index=(0,), layout=Layout(height='auto', width='auto'), options…

Button(description='Import Selected CVs', style=ButtonStyle())

[('GCV_API', 'GCV_PRD', 'ORD-CostCenterHierarchy_Normalised', 'BASE TABLE')]
[GCV_PRD].ORD-CostCenterHierarchy_Normalised_backup does not exist in the database and will need to be created once parent table has data


0 Existing records
ORD-CostCenterHierarchy_Normalised has been updated with 91616 records

Duration: 0:01:11.043310



In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.
